In [1]:
import pandas as pd
df_train = pd.read_csv('datasets/Titanic/train.csv')
df_test = pd.read_csv('datasets/Titanic/test.csv')

In [2]:
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

def preprocess(X):    
    for label in [1, 2, 6, 8, 9]:
        X[:, label] = LabelEncoder().fit_transform(X[:, label].astype(str))
    X = Imputer(missing_values="NaN", strategy="most_frequent").fit_transform(X)
#     X = OneHotEncoder(categorical_features=[1, 2, 6, 8, 9]).fit_transform(X).toarray()
    return X

X, Y = df_train.iloc[:, 2:].values, df_train.iloc[:, 1].values
X_test = df_test.iloc[:, 1:].values

X = preprocess(X)
X_test = preprocess(X_test)

X_train, X_dev, Y_train, Y_dev = train_test_split(X, Y, test_size=0.1, random_state=1234)

In [3]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_dev = sc.transform(X_dev)
X_test = sc.transform(X_test)

In [4]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

max_accuracy = 0
max_classifier = None
for n in range(1, 51):
    classifier = KNeighborsClassifier(n_neighbors=n, weights='distance')
    classifier = classifier.fit(X_train, Y_train)
    
    accuray = accuracy_score(Y_dev, classifier.predict(X_dev))
    if  accuray > max_accuracy:
        max_accuracy = accuray
        max_classifier = classifier

In [5]:
def write_csv(Y_test, filename):
    with open(filename, 'w') as f:
        f.write('PassengerId,Survived\n')
        i = 892
        for element in Y_test:
            f.write('{},{}\n'.format(i, element))
            i += 1
            
write_csv(max_classifier.predict(X_test), 'datasets/Titanic/result_knn.csv')

In [6]:
max_classifier

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=20, p=2,
           weights='distance')

In [7]:
!kaggle competitions submit -c titanic -f datasets/Titanic/result_knn.csv -m "basic knn"

Successfully submitted to Titanic: Machine Learning from Disaster